In [1]:
import json
import re
import urllib
from pprint import pprint
import time
from tqdm import tqdm

from py2neo import Node, Graph, Relationship, NodeMatcher
from py2neo.bulk import merge_nodes

import numpy as np
import pandas as pd
import wikipedia
from sklearn.metrics.pairwise import cosine_similarity

import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.matcher import Matcher
from spacy.tokens import Doc, Span, Token

print(spacy.__version__)

3.0.3


In [2]:
graph = Graph("bolt://neo4j:7687", name="neo4j", password="1234")
nodes_matcher = NodeMatcher(graph)

In [7]:
df = graph.run('MATCH (n:Node) RETURN n.name, n.description, n.node_labels, n.url, n.word_vec, n.fastrp_100').to_data_frame()
df.head()

,n.name,n.description,n.node_labels,n.url,n.word_vec,n.fastrp_100
0,oh bah mə,,[Subject],,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.10572587698698044, 0.004473589360713959, 0...."
1,american politician,,[Thing],,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,44th president,[],[],[],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,united states,"The United States of America, commonly known a...","[AdministrativeArea, Country, Thing, Place]",https://en.wikipedia.org/wiki/United_States,"[-0.06977874785661697, 0.21644160151481628, -0...","[-0.11826247721910477, -0.24348512291908264, 0..."
4,democratic party,The Democratic Party is one of the two major c...,"[Thing, Organization]",https://en.wikipedia.org/wiki/Democratic_Party...,"[0.03246232122182846, 0.12774689495563507, -0....","[-0.11826247721910477, -0.24348512291908264, 0..."
